In [ ]:
import json

file_path = r"C:\Users\HP\Desktop\vietlott-data\data\bingo18.jsonl"

results = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            record = json.loads(line)
            filtered = {
                "date": record.get("date"),
                "id": record.get("id"),
                "result": record.get("result"),
                "total": record.get("total"),
                "process_time": record.get("process_time")
            }
            results.append(filtered)
for item in results[:10]:
    print(item)
           

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\OneBim\\Desktop\\data\\vietlott-data\\data\\bingo18.jsonl'

In [ ]:
import json
from datetime import datetime

file_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18.jsonl"
output_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl"

with open(file_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        if line.strip():
            rec = json.loads(line)

            # Tách và thêm trường mới
            dt = datetime.fromisoformat(rec["process_time"])
            record = {
                "date": rec.get("date"),
                "id": rec.get("id"),
                "result": rec.get("result"),
                "total": rec.get("total"),
                "process_time": rec.get("process_time"),
                "hour": dt.hour,
                "minute": dt.minute,
                "second": dt.second
            }

            # Ghi ra file dưới dạng JSON Lines
            f_out.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ Đã lưu file JSONL mới tại: {output_path}")


✅ Đã lưu file JSONL mới tại: C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl


In [ ]:
import json
from datetime import datetime

file_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18.jsonl"
output_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl"

day_counters = {}  # lưu bộ đếm theo từng ngày

with open(file_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        if line.strip():
            rec = json.loads(line)

            # Parse datetime
            dt = datetime.fromisoformat(rec["process_time"])

            # Tính new_id theo ngày
            current_day = rec.get("date")
            if current_day not in day_counters:
                day_counters[current_day] = 1
            else:
                day_counters[current_day] += 1

            record = {
                "date": current_day,
                "id": rec.get("id"),
                "new_id": day_counters[current_day],   # thêm new_id
                "result": rec.get("result"),
                "total": rec.get("total"),
                "process_time": rec.get("process_time"),
                "hour": dt.hour,
                "minute": dt.minute,
                "second": dt.second
            }

            # Ghi ra file JSONL
            f_out.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ Đã lưu file JSONL mới với new_id tại: {output_path}")


✅ Đã lưu file JSONL mới với new_id tại: C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl


In [ ]:
import json

input_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl"
output_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_first_rows.jsonl"

with open(input_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        if line.strip():
            rec = json.loads(line)
            if rec.get("new_id") == 1:   # chỉ lấy new_id = 1
                f_out.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"✅ Đã lưu tất cả dòng có new_id = 1 vào: {output_path}")


✅ Đã lưu tất cả dòng có new_id = 1 vào: C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_first_rows.jsonl


In [ ]:
import json
import os

input_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl"
output_dir = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\split_newid"

# Tạo thư mục output nếu chưa có
os.makedirs(output_dir, exist_ok=True)

# Mở nhiều file writer theo new_id
writers = {}

with open(input_path, "r", encoding="utf-8") as f_in:
    for line in f_in:
        if line.strip():
            rec = json.loads(line)
            new_id = rec.get("new_id")
            if new_id is None:
                continue

            # Nếu chưa có writer cho new_id thì tạo mới
            if new_id not in writers:
                file_path = os.path.join(output_dir, f"bingo18_newid_{new_id}.jsonl")
                writers[new_id] = open(file_path, "w", encoding="utf-8")

            # Ghi bản ghi vào file tương ứng
            writers[new_id].write(json.dumps(rec, ensure_ascii=False) + "\n")

# Đóng tất cả file
for w in writers.values():
    w.close()

print(f"✅ Đã chia file theo từng new_id trong thư mục: {output_dir}")


✅ Đã chia file theo từng new_id trong thư mục: C:\Users\OneBim\Desktop\data\vietlott-data\data\split_newid


In [ ]:
import json
from collections import Counter, defaultdict

input_path = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\bingo18_parsed.jsonl"

# Dùng dict để gom dữ liệu theo new_id
totals_by_newid = defaultdict(list)

# Đọc file jsonl
with open(input_path, "r", encoding="utf-8") as f_in:
    for line in f_in:
        if line.strip():
            rec = json.loads(line)
            new_id = rec.get("new_id")
            total = rec.get("total")
            if new_id is not None and total is not None:
                totals_by_newid[new_id].append(total)

# Tìm giá trị total phổ biến nhất cho từng new_id
result = {}
for new_id, totals in totals_by_newid.items():
    counter = Counter(totals)
    most_common_total, freq = counter.most_common(1)[0]
    result[new_id] = {
        "most_common_total": most_common_total,
        "count": freq
    }

# In kết quả
for nid in sorted(result.keys()):
    print(f"new_id {nid}: total {result[nid]['most_common_total']} xuất hiện {result[nid]['count']} lần")


new_id 1: total 10 xuất hiện 36 lần
new_id 2: total 10 xuất hiện 36 lần
new_id 3: total 12 xuất hiện 43 lần
new_id 4: total 9 xuất hiện 38 lần
new_id 5: total 12 xuất hiện 40 lần
new_id 6: total 9 xuất hiện 35 lần
new_id 7: total 10 xuất hiện 45 lần
new_id 8: total 11 xuất hiện 34 lần
new_id 9: total 11 xuất hiện 44 lần
new_id 10: total 12 xuất hiện 37 lần
new_id 11: total 11 xuất hiện 44 lần
new_id 12: total 11 xuất hiện 34 lần
new_id 13: total 10 xuất hiện 38 lần
new_id 14: total 11 xuất hiện 45 lần
new_id 15: total 9 xuất hiện 36 lần
new_id 16: total 13 xuất hiện 40 lần
new_id 17: total 9 xuất hiện 34 lần
new_id 18: total 13 xuất hiện 35 lần
new_id 19: total 11 xuất hiện 40 lần
new_id 20: total 10 xuất hiện 41 lần
new_id 21: total 9 xuất hiện 39 lần
new_id 22: total 12 xuất hiện 37 lần
new_id 23: total 10 xuất hiện 38 lần
new_id 24: total 10 xuất hiện 39 lần
new_id 25: total 12 xuất hiện 39 lần
new_id 26: total 11 xuất hiện 35 lần
new_id 27: total 9 xuất hiện 45 lần
new_id 28: total

In [ ]:
import json
import os
from collections import Counter

input_dir = r"C:\Users\OneBim\Desktop\data\vietlott-data\data\split_newid"

for file_name in os.listdir(input_dir):
    if file_name.startswith("bingo18_newid_") and file_name.endswith(".jsonl"):
        file_path = os.path.join(input_dir, file_name)

        # Đọc file
        results = []
        with open(file_path, "r", encoding="utf-8") as f_in:
            for line in f_in:
                if line.strip():
                    rec = json.loads(line)
                    res = tuple(rec.get("result", []))  # dùng tuple để đếm
                    if res:
                        results.append(res)

        counter = Counter(results)
        duplicates = {res: cnt for res, cnt in counter.items() if cnt > 1}

        print(f"\n📂 File: {file_name}")
        if duplicates:
            # result xuất hiện nhiều nhất
            most_common_res, freq = counter.most_common(1)[0]
            print(f"🔎 result xuất hiện nhiều nhất: {list(most_common_res)} ({freq} lần)")
            print("👉 Các result trùng khác:")
            for res, cnt in duplicates.items():
                print(f"   {list(res)} -> {cnt} lần")
        else:
            print("✅ Không có result nào trùng lặp trong file này")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\OneBim\\Desktop\\data\\vietlott-data\\data\\split_newid'